In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stat
import statsmodels.api as sm

from mpl_toolkits import mplot3d

pd.set_option('display.max_columns', 9999)

%matplotlib notebook

In [2]:
med = pd.read_csv('eicu-collaborative-research-database-2.0/medication.csv.gz')

/Users/adeshkadambi/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def opioids (x):
    
    opiods = ['hydromorphone','hydrocodone','morphine','oxycodone','oxymorphone','codeine','fentanyl','meperidine',
          'tramadol','carfentanil','percocet','norco']

    for i in opiods:
        if i in str(x).lower():
            return 1
        
    return 0

In [4]:
def nsaids (x):
    
    non_op = ['acetaminophen','amitriptyline', 'doxepin', 'imipramine', 'desipramine', 'nortriptyline',
         'ibuprofen', 'naproxen', 'diclofenac', 'piroxicam', 'sulindac', 'indomethacin', 'ketorolac', 'meloxicam', 
          'celecoxib', 'ketoprofen', 'oxaprozin', 'toradol', 'valdecoxib', 'bextra', 'rofecoxib', 'vioxx', 'gabapentin', 
          'neurontin', 'cyclobenzaprine', 'duloxetine', 'cymbalta', 'pregabalin', 'lyrica', 'venlafaxine', 'effexor', 
          'tylenol', 'voltaren', 'naprosyn', 'paracetamol', 'aspirin']
    
    for j in non_op:
        if j in str(x).lower():
            return 1

    return 0

In [5]:
med['opioid'] = med.drugname.apply(lambda x: opioids(x))
med['nsaid'] = med.drugname.apply(lambda x: nsaids(x))

med = med[med.drugordercancelled == 'No']

In [6]:
cols_drop = ['medicationid','drugorderoffset','drugivadmixture','drugordercancelled','drughiclseqno',
            'routeadmin','loadingdose','prn','gtc']

med.drop(columns=cols_drop, inplace=True)
med = med[(med.opioid == 1) | (med.nsaid == 1)]

In [7]:
med.head()

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,drugstopoffset,opioid,nsaid
2,141168,1386,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2390,0,1
15,141194,2109,1 ML - HYDROMORPHONE HCL 1 MG/ML IJ SOLN,0.4 3,Q3H PRN,12622,1,0
33,141203,-339,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2414,0,1
37,141203,1873,MORPHINE INJ,2 3,Q15 Min PRN,4189,1,0
51,141227,-843,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,1744,0,1


In [ ]:
mg = med.groupby(['patientunitstayid']).agg({'opioid': lambda x:sum(x), 'nsaid': lambda x:sum(x)})
mg[mg != 0] = 1

In [ ]:
def op_find (x, arr):
    try:
        return arr['opioid'][x]
    except:
        return np.nan
    
def nsaid_find (x, arr):
    try:
        return arr['nsaid'][x]
    except:
        return np.nan

In [ ]:
pts = pd.read_csv('eicu-collaborative-research-database-2.0/patient.csv.gz')
hos = pd.read_csv('eicu-collaborative-research-database-2.0/hospital.csv.gz')

In [ ]:
pts['opioid'] = pts['patientunitstayid'].apply(lambda x: op_find(x, mg))
pts['nsaid'] = pts['patientunitstayid'].apply(lambda x: nsaid_find(x, mg))

pts = pd.merge(pts, hos, on=['hospitalid'], how = 'left')

In [ ]:
pts = pts[pts.gender != 'Unknown']
pts = pts[pts.gender != 'Other']

In [ ]:
pts['apacheadmissiondx'].fillna(value = 'N/A', inplace = True)
pts['hospitaladmitsource'].fillna(value = 'Unknown', inplace = True)

def age (x):
    if x == '> 89':
        return 90
    else:
        return int(x)

pts['age'].fillna(value = 0, inplace = True)
pts['age'] = pts['age'].apply(lambda x: age(x))

In [ ]:
pts.opioid.fillna(0, inplace = True)
pts.nsaid.fillna(0, inplace = True)

In [ ]:
pts.opioid.replace(1.0,True,inplace=True)
pts.opioid.replace(0.0,False,inplace=True)

pts.nsaid.replace(1.0,True,inplace=True)
pts.nsaid.replace(0.0,False,inplace=True)

In [ ]:
pts['painmeds'] = pts.opioid|pts.nsaid

In [ ]:
pts.head()

In [ ]:
apv = pd.read_csv('eicu-collaborative-research-database-2.0/apachePredVar.csv.gz')
apv = apv[['patientunitstayid', 'bedcount']]
pts = pd.merge(pts, apv, on=['patientunitstayid'], how='left')

In [ ]:
# hospital odds ratios

pts_hosp = pts[['patientunitstayid','ethnicity','hospitalid','wardid','region','numbedscategory',
                'bedcount','teachingstatus','opioid','nsaid','painmeds']]
pts_hosp.head()

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='gender',y='painmeds',hue='ethnicity',data=pts, estimator=np.mean)
plt.xlabel('Patient Gender')
plt.ylabel('Proportion Receiving Pain Medications')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='gender',y='nsaid',hue='ethnicity',data=pts, estimator=np.mean)
plt.xlabel('Patient Gender')
plt.ylabel('Proportion Receiving NSAIDs')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='gender',y='opioid',hue='ethnicity',data=pts, estimator=np.mean)
plt.xlabel('Patient Gender')
plt.ylabel('Proportion Receiving Opioids')

In [ ]:
ct_eth = pd.crosstab(pts.painmeds, pts.ethnicity, margins= True)
ct_eth

In [ ]:
fobs = [14170, 1933, 99411, 4763, 1214, 6736]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

In [ ]:
ct_op = pd.crosstab(pts.opioid, pts.ethnicity, margins= True)
ct_op

In [ ]:
fobs = [10510,1487,75398,3175,989,5277]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

In [ ]:
ct_ns = pd.crosstab(pts.nsaid, pts.ethnicity, margins= True)
ct_ns

In [ ]:
fobs = [12165,1688,83939,4151,868,5634]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

In [ ]:
dxs = pd.read_csv('eicu-collaborative-research-database-2.0/diagnosis.csv.gz')

# adding icd9 codes
dxs = dxs.drop(columns=['diagnosisid', 'activeupondischarge', 'diagnosisoffset', 'diagnosisstring'])
dxs.dropna(inplace=True)

# merge
dxs = dxs.groupby('patientunitstayid')['icd9code'].apply(list).reset_index(name='icd9codes')
pts = pd.merge(pts, dxs, on=['patientunitstayid'], how='left')

In [ ]:
cpg = pd.read_csv('eicu-collaborative-research-database-2.0/carePlanGeneral.csv.gz')

# adding ordered protocols
cpg = cpg[cpg.cplgroup == 'Ordered Protocols']
cpg.drop(columns=['cplgeneralid', 'activeupondischarge', 'cplitemoffset', 'cplgroup'], inplace = True)
cpg.drop_duplicates(inplace=True)

# merge
cpg = cpg.groupby('patientunitstayid')['cplitemvalue'].apply(list).reset_index(name='orderedprotocols')
pts = pd.merge(pts, cpg, on=['patientunitstayid'], how = 'left')

In [ ]:
nsc = pd.read_csv('eicu-collaborative-research-database-2.0/nurseCharting.csv.gz')

# adding pain scores
nsc.drop(['nursingchartid', 'nursingchartoffset', 
          'nursingchartcelltypecat', 'nursingchartcelltypevallabel'], axis=1, inplace=True)
nsc = nsc[nsc['nursingchartcelltypevalname'] == 'Pain Score']

In [ ]:
valid = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

nsc['nursingchartvalue'] = nsc['nursingchartvalue'].apply(lambda x: float(x))
nsc = nsc[nsc['nursingchartvalue'].isin(valid)]

nsc['nursingchartvalue'] = nsc['nursingchartvalue'].apply(lambda x: int(x))
nsc.drop(['nursingchartcelltypevalname'], axis=1, inplace=True)

In [ ]:
max_pain = nsc.groupby('patientunitstayid').max()
mean_pain = nsc.groupby('patientunitstayid').mean()

In [ ]:
def pain_val (x, arr):
    try:
        return round(arr['nursingchartvalue'][x])
    except:
        return np.nan

pts['max_pain'] = pts['patientunitstayid'].apply(lambda x: pain_val(x, max_pain))
pts['mean_pain'] = pts['patientunitstayid'].apply(lambda x: pain_val(x, mean_pain))

In [ ]:
# Two way T test for Native American no-analgesia vs all other Ethnicities no-analgesia PAIN SCORES
npts = pts[pts.painmeds == False]

natpts = npts[npts.ethnicity == 'Native American']
othpts = npts[npts.ethnicity != 'Native American']

print(stat.ttest_ind(natpts.mean_pain.dropna(), othpts.mean_pain.dropna()))

**Difference in pain scores between Native American no-analgesia vs all other Ethnicities no-analgesia not significant**

In [ ]:
# pts['native'] = pts.ethnicity

# pts.native.replace('African American', 'Non-native', inplace=True)
# pts.native.replace('Asian', 'Non-native', inplace=True)
# pts.native.replace('Caucasian', 'Non-native', inplace=True)
# pts.native.replace('Hispanic', 'Non-native', inplace=True)
# pts.native.replace('Other/Unknown', 'Non-native', inplace=True)
# pts.native.replace('Native American', 'Native', inplace=True)

In [ ]:
ct_eth = pd.crosstab(pts.painmeds, pts.ethnicity, margins= True)
ct_eth

In [ ]:
fobs = [14550, 1953, 100655, 4770, 1218, 6796]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

_____
#### Temporal Analysis

In [8]:
med.head(20)

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,drugstopoffset,opioid,nsaid
2,141168,1386,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2390,0,1
15,141194,2109,1 ML - HYDROMORPHONE HCL 1 MG/ML IJ SOLN,0.4 3,Q3H PRN,12622,1,0
33,141203,-339,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2414,0,1
37,141203,1873,MORPHINE INJ,2 3,Q15 Min PRN,4189,1,0
51,141227,-843,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,1744,0,1
62,141229,-100,MORPHINE INJ,4 3,Q30 Min PRN,3628,1,0
92,141244,-10800,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,-9715,0,1
94,141244,-11713,MORPHINE INJ,4 3,Q30 Min PRN,-11517,1,0
96,141244,-9285,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,8416,0,1
104,141260,30,OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS,1 5002,Q6H PRN,1999,1,1


In [ ]:
med['duration_min'] = med.drugstopoffset - med.drugstartoffset
med.drop(columns=['drugstopoffset'], inplace=True)

In [ ]:
med.dropna(inplace=True)

In [ ]:
opi = med[med.opioid==1]

In [ ]:
opi.head()

In [ ]:
opi = opi[opi.frequency != 'Pyxis']

In [ ]:
dict(opi.frequency.value_counts())

In [ ]:
dict(opi.dosage.value_counts())

In [ ]:
import re

In [ ]:
t = '25-200 mcg/hr'

In [ ]:
regex = re.findall(r"[-+]?\d*\.\d+|\d+", t)
regex = [float(x) for x in regex]
regex

if len(regex) > 1:
    print( regex[-1] )
else:
    print( regex[0] )

In [ ]:
def cases (x):
    
    try:
        if 'mcg/hr' in x.lower():
            regex = re.findall(r"[-+]?\d*\.\d+|\d+", x)
            regex = [float(i) for i in regex]

            if len(regex) > 1:
                return str(regex[-1]/1000) + ' hr'
            else:
                return str(regex[0]/1000) + ' hr'
    
        elif 'mg' in x.lower():
            regex = re.findall(r"[-+]?\d*\.\d+|\d+", x)
            regex = [float(i) for i in regex]

            if len(regex) > 1:
                return regex[-1]
            else:
                return regex[0]

        elif 'mcg' in x.lower():
            regex = re.findall(r"[-+]?\d*\.\d+|\d+", x)
            regex = [float(i) for i in regex]

            if len(regex) > 1:
                return regex[-1]/1000
            else:
                return regex[0]/1000
        else:
            return x.lower()
    except:
        return x.lower()

In [ ]:
cases(t)

In [ ]:
opi['dos_mg'] = opi.dosage.apply(lambda x: cases(x))

In [ ]:
opi

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
opi.dos_mg.value_counts().index